# Example: Get, inspect, change, and put back a instrument method with a binary solvent manager

Here outlines how to interact with instrument methods and module methods.

- Finding and getting a instrument method.
- Inspecting what types of module methods are in the instrument method.
- Inspecting the instrument parameters in the method, including column temperature and gradient table.
- Changing instrument parameters and posting the new method.

Get API Address

In [2]:
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore") # Required to suppress warnings from requests 

# get api address from .env file
load_dotenv("../.env_vars")
EMPOWER_API_ADDRESS = os.getenv("EMPOWER_API_ADDRESS_PRD")

# Finding and getting a instrument method

We first find the list of all instrument methods, select one, and get that from Empower:

In [4]:
from OptiHPLCHandler import EmpowerHandler
import pprint

# Create an instance of the EmpowerHandler class
handler = EmpowerHandler(project="WebAPI_test", address=EMPOWER_API_ADDRESS)
handler.connection.default_get_timeout = 120
handler.connection.default_post_timeout = 120

In [3]:
# Get the list of methods, select one, and get the method details
with handler:
    method_list = handler.GetMethodList()  # Get the list of instrument methods
    method_name = method_list[0]  # Select the first method
    print(method_name)
    full_method = handler.GetInstrumentMethod(method_name)

@BSM_PDA_ON_Template


# Inspecting the instrument method

We can now look into the module methods in the instrument method. 

In [4]:
pp = pprint.PrettyPrinter(indent=2)

print(full_method)
print(f"Valve positions: {full_method.module_method_list[-1].valve_position}")
# Printing the valve position for the solvent manager module method in the list.
# Notice that we do not need to know the tag name to print the valve position.
print("\n\nStart of gradient table:\n")
pp.pprint(full_method.gradient_table[0:2])
# Printing the first two entries gradient table.
# Notice that we do not need to know the tag names to print the gradient table

EmpowerInstrumentMethod with 4 module methods of types PDAMethod, SampleManagerMethod, ColumnManagerMethod, BSMMethod
Valve positions: ['A1', 'B2']


Start of gradient table:

[ { 'CompositionA': '50.0',
    'CompositionB': '50.0',
    'Curve': 'Initial',
    'Flow': '0.300',
    'Time': 'Initial'},
  { 'CompositionA': '0.0',
    'CompositionB': '100.0',
    'Curve': '6',
    'Flow': '0.300',
    'Time': '2.00'}]


In [5]:
print("\n\nDetector settings:")
# The list of detectors found in the instrument method
print(full_method.detector_method_list)

# Check if lamp enabled
print("\nLamp status:")
print(full_method.detector_method_list[0].lamp_enabled)

# Check the channel settings
print("\nChannel settings:")
pprint.pprint(full_method.detector_method_list[0].channels)
print(full_method.detector_method_list[0].wavelengths)



Detector settings:

Lamp status:
True

Channel settings:
[PDAChannel(wavelength1='222', wavelength2='498', resolution='Resolution_48', datamode='DataModeAbsorbance_0', ratio2dminimumau='0.01')]
['222']


Note that most of the methods are simply EmpowerModuleMethods. That is the generic
type for module methods that aren't specifically accounted for. At the moment, that
is every type of method expect:
 - Solvent manager methods (BSMMethod and QSMMethod), which have the special properties `valve_position` and `gradient_table`.
 - Column oven methods (SampleManagerMethod), which have the special property `column_temperature`.

All of the special properties can be accessed as set from the `EmpowerInstrumentMethod`.
Getting `EmpowerInstrumentMethod.column_temperature` will produce an error if the
instrument method controls several column ovens (e.g. one in a sample manager and one in
a column manager), and the column ovens have different temperatures. Setting 
`EmpowerInstrumentMethod.column_temperature` will set the temperature for all column
ovens if there are any, and produce an error otherwise.

Here is an example for a quaternary system:

In [6]:
import pprint

from OptiHPLCHandler import EmpowerHandler

# Create an instance of the EmpowerHandler class
handler = EmpowerHandler(project="WebAPI_test", address=EMPOWER_API_ADDRESS)
handler.connection.default_get_timeout = 120
handler.connection.default_post_timeout = 120

# Get the list of methods, select one, and get the method details
with handler:
    method_list = handler.GetMethodList()  # Get the list of instrument methods
    method_name = method_list[2]  # Select the first method
    print(method_name)
    full_method = handler.GetInstrumentMethod(method_name)

c:\Users\slcb\Desktop\Programming\OptiHPLCHandler\.env\lib\site-packages\urllib3\connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host ***REMOVED***. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


@BSM_PDA_Template_1_0C


In [7]:
pp = pprint.PrettyPrinter(indent=2)

print(full_method)
print(f"Valve positions: {full_method.module_method_list[3].valve_position}")
# Printing the valve position for the solvent manager module method in the list.
print("\n\nStart of gradient table:\n")
pp.pprint(full_method.gradient_table[0:2])
# Printing the first two entries gradient table.

EmpowerInstrumentMethod with 4 module methods of types PDAMethod, SampleManagerMethod, ColumnManagerMethod, BSMMethod
Valve positions: ['A1', 'B2']


Start of gradient table:

[ { 'CompositionA': '50.0',
    'CompositionB': '50.0',
    'Curve': 'Initial',
    'Flow': '0.300',
    'Time': 'Initial'},
  { 'CompositionA': '0.0',
    'CompositionB': '100.0',
    'Curve': '6',
    'Flow': '0.300',
    'Time': '1.00'}]


# Changing instrument parameters and posting the new method.

We can also change the values of the instrument method, and post the changed method back
to Empower, so we can use it to analyse samples.

Remember to give the method a new name before posting it to Empower. OptiHPLCHandler
does not allow changing methods in Empower, only creating new ones.

In [ ]:
gradient_table = full_method.gradient_table  # Get the gradient table
for step in gradient_table:
    step["Flow"] = 0.5  # Set the flow to 0.5 mL/min for all steps
gradient_table[1]["Time"] = 5 / 3  # Set the time for the second step to 5/3 minutes.
# Notice the warning that 1.666666667 minutes is rounded to 1.667, since Empower will
# misinterpret values with too many decimals.
full_method.gradient_table = (
    gradient_table  # Set the gradient table to the updated gradient table
)
full_method.valve_position = [
    "A2",
    "B1",
]  # Set the valve position to A2 and B1. You can also set only one of the valves.
full_method.method_name = "New Method Name 1"  # Set the method name
with handler:
    handler.PostInstrumentMethod(full_method)  # Post the updated method to Empower

In [11]:
from OptiHPLCHandler.empower_detector_module_method import PDAChannel

In [ ]:
# changing the wavelength of a single wavelength detector using channels attribute
print("\n\nDetector settings:")
pprint.pprint(full_method.detector_method_list[0].channels)
channel = PDAChannel(wavelength1=111)
full_method.detector_method_list[0].channels = [channel]
print("\n\nNew Detector settings:")
pprint.pprint(full_method.detector_method_list[0].channels)



Detector settings:
[PDAChannel(wavelength1='214', wavelength2='498', resolution='Resolution_48', datamode='DataModeAbsorbance_0', ratio2dminimumau='0.01')]


New Detector settings:
[PDAChannel(wavelength1='111', wavelength2='254', resolution='Resolution_48', datamode='DataModeAbsorbance_0', ratio2dminimumau='0.01')]


In [18]:
# changing the wavelength of a single wavelength detector using wavelength attribute
print("Detector settings:")
pprint.pprint(full_method.detector_method_list[0].wavelengths)
wavelengths = [222, 333, 444]
full_method.detector_method_list[0].wavelengths = wavelengths
print("\n\nNew Detector settings:")
pprint.pprint(full_method.detector_method_list[0].wavelengths)

Detector settings:
['222', '333', '444']


New Detector settings:
['222', '333', '444']


In [25]:
from OptiHPLCHandler.empower_detector_module_method import PDASpectralChannel

In [26]:
# changing the wavelength of a single wavelength detector
print("Detector settings:")
pprint.pprint(full_method.detector_method_list[0].spectral_channel)
spectral = PDASpectralChannel(start_wavelength=333, end_wavelength=444)
full_method.detector_method_list[0].spectral_channel = spectral
print("\n\nNew Detector settings:")
pprint.pprint(full_method.detector_method_list[0].spectral_channel)

Detector settings:
PDASpectralChannel(start_wavelength='111', end_wavelength='222', resolution='Resolution_12')


New Detector settings:
PDASpectralChannel(start_wavelength='333', end_wavelength='444', resolution='Resolution_12')


In [27]:
# changing the wavelength of a single wavelength detector
print("Detector settings:")
pprint.pprint(full_method.detector_method_list[0].spectral_wavelengths)
full_method.detector_method_list[0].spectral_wavelengths = [
    {
        "Start Wavelength": 111,
        "End Wavelength": 222,
    },
]
print("\n\nNew Detector settings:")
pprint.pprint(full_method.detector_method_list[0].spectral_wavelengths)

Detector settings:
[{'End Wavelength': '444', 'Start Wavelength': '333'}]


New Detector settings:
[{'End Wavelength': '222', 'Start Wavelength': '111'}]


In [ ]:
with handler:
    handler.PostInstrumentMethod(full_method)  # Post the updated method to Empower

More info on detectors in example_detectors.ipynb

# MethodSet methods

In order to run an instrument method, we need a methodset method that includes it.

Let's start by logging in, getting the list of all methodset methods present, and look
at one of them

In [17]:
with handler:
    methodset_method_list = handler.GetMethodList("MethodSet")
    print(methodset_method_list)
    methodsset_method = handler.GetMethodSetMethod(methodset_method_list[0])
methodsset_method

['@BSM_PDA_Template', '@BSM_PDA_Template_1_0C', '@BSM_PDA_Template_1_0pct', '@BSM_PDA_Template_2_0pc_2_0pct', '@BSM_PDA_Template_2_0pc_3_0pct', '@BSM_PDA_Template_iso2_5min', '@BSM_PDA_Template_iso4_5min', '@BSM_PDA_Template_ramp', '@BSM_PDA_Template_ramp_ramp', '@BSM_TUV_Template', '@QSM_PDA_Template', '20240409_slcb', '20240409_slcb_test', '20240409_slcb_test_1_0pct', '20240409_slcb_test_2_5C', '20240409_slcb_test_iso_30_0m', '20240409_slcb_test_iso_60_0m', '20240409_slcb_test_m1_0pct', '20240409_slcb_test_m2_5C', '20240409_slcb_test_ramp', '20240410_test', '20240410_test_1_0pct', '20240410_test_2_5C', '20240410_test_cond_10m', '20240410_test_iso_30_0m', '20240410_test_iso_60_0m', '20240410_test_m1_0pct', '20240410_test_m2_5C', '20240410_test_ramp', '20240416_test', '20240416_test_1_0pct', '20240416_test_2_5C', '20240416_test_cond_10m', '20240416_test_iso_15m_0', '20240416_test_iso_30m_0', '20240416_test_low', '20240416_test_m1_0pct', '20240416_test_m2_5C', '20240416_test_ramp', '202

{'name': '@BSM_PDA_Template',
 'isLocked': False,
 'id': 1463,
 'version': 1,
 'date': '2023-11-10T08:18:55',
 'comments': '',
 'modifiedBy': 'SLCB',
 'revisionHistory': [{'id': 1463,
   'version': 1,
   'modificationDate': '10-Nov-2023 08:18:55 CET',
   'user': 'SLCB',
   'comment': "Created method '@BSM_PDA_Template'."}],
 'instrumentMethod': '@BSM_PDA_Template'}

We can now create a simple MethodSet method

In [18]:
method_set_method = {
    "name": "New Methodset Method 1",
    "instrumentMethod": full_method.method_name,
}
with handler:
    handler.PostMethodSetMethod(method_set_method)

We can also specify more about the methodset method - Export methods, and default processing and reporting methods

In [ ]:
with handler:
    processing_method_list = handler.GetMethodList("Processing")
    report_method_list = handler.GetMethodList("Report")
    method_set_method = {
        "name": "Complex new methodset method 1",
        "instrumentMethod": full_method.method_name,
        "defaultProcessingMethod": processing_method_list[0],
        "defaultReportingMethod": report_method_list[0],
    }
    handler.PostMethodSetMethod(method_set_method)

Copying Methods

In [4]:
# Get the list of methods, select one, and get the method details
with handler:
    method_list = handler.GetMethodList(
        method_type="InstrumentMethod"
    )  # Get the list of instrument methods
    method_name = method_list[-1]  # Select the first method
    print(method_name)
    full_method = handler.GetInstrumentMethod(method_name)

test20240627_1


In [7]:
print(full_method.method_name)  # original method name
full_method.method_name = "New Method Name Original"  # Set the method name
print(full_method.method_name)  # new method name

New Method Name 1
New Method Name Original


In [8]:
full_method_copy = full_method.copy()
print(full_method_copy.method_name)  # new method name
full_method_copy.method_name = "New Method Name Copy"  # Set the method name
print(full_method_copy.method_name)  # new method name
print(full_method.method_name)  # show other method not changed

New Method Name Original
New Method Name Copy
New Method Name Original
